In this notebook I intend to change how directories are handled. Currently FRUST can only save a calculation directory and all files from the calculation folder. Also a scratch folder is not used, which can bslow down calculations.

Intend behavioral changes
- [x] Save only needed files
- [ ] Use scratch for calculation directory.

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from tooltoad.chemutils import xyz2ac
from tooltoad.orca import orca_calculate

inp = """3
water
O     -5.099200    3.059520    0.000003
H     -4.747591    3.911975   -0.228455
H     -5.982743    3.160594    0.334122"""

atoms, coords = xyz2ac(inp)

result = orca_calculate(atoms, coords, options={"HF": None, "STO-3G": None, "Opt": None, "Freq": None}, save_dir="noob", save_files=["orca.out", "*.xyz"], n_cores=1)

In [3]:
import pandas as pd
from frust.transformers import transformer_mols
from frust.embedder import embed_mols
from rdkit import Chem

smi = "CN1C=CC=C1"
m = Chem.MolFromSmiles(smi)
mol_dict = {"mol": m}
mols_dict_embedded = embed_mols(mol_dict, n_confs=1)

In [4]:
from frust.stepper import Stepper
step = Stepper([smi], save_output_dir=False, n_cores=2, memory_gb=2, debug=False)
df0 = step.build_initial_df(mols_dict_embedded)
df0

2025-07-14 10:44:27 WARNING frust.stepper: 




,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,mol,mol,<NA>,None,0,None,"[C, N, C, C, C, C, H, H, H, H, H, H, H]","[(-1.885251124434566, 0.12570471949082884, 0.0...",None


In [5]:
df1 = step.orca(df0, options={"HF": None, "STO-3G": None})

2025-07-14 10:45:19 INFO  frust.stepper: [orca-HF-STO-3G] row 0 (mol)…


In [22]:
import submitit
import os

DEBUG           = False
N_CORES         = 1
MEM_GB          = 2
TIMEOUT_MIN     = 14400


def orcish_warrior():
    try:
        scratch = os.environ["SCRATCH"]
    except:
        scratch = "."
    os.makedirs(scratch, exist_ok=True)

    print(f"[DEBUG] Using scratch dir: {scratch}")

    res = orca_calculate(
        atoms, coords,
        options={"HF": None, "STO-3G": None},
        scr=scratch,
        n_cores=1,
        save_dir="noob",
        save_files=["orca.out"]
    )
    return res


executor = submitit.AutoExecutor("logs/test", cluster="slurm")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

executor.update_parameters(slurm_job_name="test_job")
executor.submit(orcish_warrior)

SlurmJob<job_id=53002686, task_id=0, state="UNKNOWN">